In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pylab as plt
from mpl_toolkits.axes_grid1 import ImageGrid, AxesGrid, make_axes_locatable, SubplotDivider

In [ ]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../src'))

from figure_presets import *
from plotting_functions import *

from adaptive_response.adaptive_threshold import AdaptiveThresholdNumeric
from utils.numba.tools import random_seed

In [ ]:
Nl = 256
Nr = 32
alpha = 1.3
s = 0.1 * Nl
width = 1

In [ ]:
parameters = {'c_distribution': 'log-normal'}

model = AdaptiveThresholdNumeric(Nl, Nr, parameters=parameters)
model.threshold_factor = alpha
model.choose_commonness('const', mean_mixture_size=s)
model.c_means = 1
model.c_vars = .1
model.choose_sensitivity_matrix('log-normal', mean_sensitivity=1, width=width)


init_state = model.parameters['initialize_state']
init_state['c_mean'] = 'exact'
init_state['c_var'] = 'exact'
init_state['correlations'] = 'exact'

In [ ]:
darkblue = "#02324F"
darkorange = "#914301"

random_seed(2)

with figure_file(
        'histogram_mixture_size.pdf',
        fig_width_pt=200., crop_pdf=False, legend_frame=False,
        transparent=True, post_process=False,
        ) as fig:
    
    # get calculated bounds
    ax = plt.gca()
    bounds = ax.get_position().bounds
    plt.delaxes(ax)

    grid = AxesGrid(fig, bounds,  
                    nrows_ncols=(2, 2),
                    axes_pad=0.1,  # pad between axes in inch.
                    share_all=False)    

    for ax_k, s in enumerate((8, 64)):
        # add the histogram
        ax = grid[2*ax_k]
        ax.set_aspect(6/s)

        model.choose_commonness('const', mean_mixture_size=s)

        en = next(model._sample_excitations(1))

        xs = np.arange(len(en)) + 1
        bars = ax.bar(xs - 0.5, en, width=1, edgecolor='none', lw=0, color=COLOR_BLUE)

        ax.axhline(alpha * en.mean(), color=COLOR_RED)

        for i in np.flatnonzero(en > alpha * en.mean()):
            bars[i].set_color(darkblue)

        ax.set_xlim(0.5, len(en) + 0.5)
        ax.set_ylim(0, 2.4*s)
        #ax.set_yticks(np.arange(0, 2.5*s, s))
        if s == 8:
            ax.set_yticks(np.arange(0, 16, 5))
        elif s == 64:
            ax.set_yticks(np.arange(0, 140, 50))
        ax.set_xticks(np.arange(10, 35, 10))

        ax.set_ylabel('$e_n$')

        # add histogram
        axHist = grid[2*ax_k + 1]
        axHist.set_aspect(0.07/s)

        bins, height = np.linspace(0, 2.2*s, num=32, retstep=True)
        bars = np.zeros(len(bins) + 1)
        for en in model._sample_excitations(10000): # 10000
            for e in en:
                bars[np.searchsorted(bins, e)] += 1

        en_mean = model.excitation_statistics_estimate()['mean'].mean()
        en_thresh = alpha*en_mean
        bars /= bars.sum()        

        idx = np.flatnonzero(bins > en_thresh)[0] - 1
        #axHist.barh(bins, bars[:-1], height=height, edgecolor='none', lw=0, color=COLOR_BLUE)
        axHist.barh(bins[:idx], bars[:idx], height=height, edgecolor='none', lw=0, color=COLOR_BLUE)
        axHist.barh(bins[idx:], bars[idx:-1], height=height, edgecolor='none', lw=0, color=darkblue)
        axHist.axhline(en_thresh, color=COLOR_RED)
        
        axHist.set_xlim(0, 0.19)
        axHist.set_xticks([0, 0.1])
        axHist.set_xticklabels(["0", "0.1"])

    ax.set_xlabel('Channel $n$');
    axHist.set_xlabel('Frequency');
    
print("Finished")